In [ ]:
# Script para converter arquivos TXT, CSV ou XLSX em SHA-256
# Autor: Suelio Lima
# Email de contato: suelio@gmail.com
# GitHub: https://github.com/suelio
# Versão: 1.0

# Licença: MIT
# Permissão é concedida, gratuitamente, a qualquer pessoa que obtenha uma cópia
# deste software e arquivos de documentação associados (o "Software"), para lidar
# no Software sem restrição, incluindo, sem limitação, os direitos de usar, copiar,
# modificar, mesclar, publicar, distribuir, sublicenciar e/ou vender cópias do Software,
# e para permitir que as pessoas a quem o Software é fornecido o façam, sujeito às
# seguintes condições:
# O aviso de direitos autorais acima e este aviso de permissão devem ser incluídos em
# todas as cópias ou partes substanciais do Software.
# O SOFTWARE É FORNECIDO "COMO ESTÁ", SEM GARANTIA DE QUALQUER TIPO, EXPRESSA OU
# IMPLÍCITA, INCLUINDO, MAS NÃO SE LIMITANDO ÀS GARANTIAS DE COMERCIALIZAÇÃO,
# ADEQUAÇÃO A UM DETERMINADO FIM E NÃO VIOLAÇÃO. EM NENHUMA HIPÓTESE OS AUTORES OU
# TITULARES DOS DIREITOS AUTORAIS SERÃO RESPONSÁVEIS POR QUALQUER REIVINDICAÇÃO,
# DANOS OU OUTRA RESPONSABILIDADE, SEJA EM UMA AÇÃO DE CONTRATO, ATO ILÍCITO OU OUTRA,
# DECORRENTE DE, FORA DE OU EM CONEXÃO COM O SOFTWARE OU O USO OU OUTRAS NEGOCIAÇÕES NO
# SOFTWARE.

# Instalação das bibliotecas necessárias (caso não estejam instaladas)
# No Google Colab, essas bibliotecas já estão instaladas por padrão, então esses comandos podem ser ignorados.
# Se estiver usando outro ambiente, descomente as linhas abaixo para instalar as dependências.
# !pip install pandas openpyxl

# Importação das bibliotecas necessárias
import hashlib  # Para calcular o hash SHA-256
import pandas as pd  # Para manipulação de dados (leitura e escrita de arquivos CSV, XLSX, etc.)
import os  # Para manipulação de caminhos de arquivos

# Função para calcular o hash SHA-256 de um texto
def calcular_sha256(texto):
    """
    Calcula o hash SHA-256 de um texto fornecido.

    Parâmetros:
    texto (str): O texto a ser convertido em hash.

    Retorna:
    str: O hash SHA-256 do texto.
    """
    return hashlib.sha256(texto.encode()).hexdigest()

# Função para verificar se o arquivo tem uma extensão válida (.txt, .csv, .xlsx)
def verificar_arquivo(nome_arquivo):
    """
    Verifica se o arquivo tem uma extensão válida (.txt, .csv, .xlsx).

    Parâmetros:
    nome_arquivo (str): O nome do arquivo a ser verificado.

    Retorna:
    bool: True se a extensão for válida, False caso contrário.
    """
    extensao = os.path.splitext(nome_arquivo)[1].lower()  # Extrai a extensão do arquivo
    if extensao not in ['.txt', '.csv', '.xlsx']:  # Verifica se a extensão é válida
        print("Erro: O arquivo deve ser do tipo TXT, CSV ou XLSX.")
        return False
    return True

# Função para verificar se o arquivo contém apenas uma coluna
def verificar_coluna_unica(df):
    """
    Verifica se o DataFrame contém apenas uma coluna.

    Parâmetros:
    df (pd.DataFrame): O DataFrame a ser verificado.

    Retorna:
    bool: True se o DataFrame tiver apenas uma coluna, False caso contrário.
    """
    if len(df.columns) != 1:  # Verifica se o número de colunas é diferente de 1
        print("Erro: O arquivo deve conter apenas uma coluna.")
        return False
    return True

# Função principal do script
def main():
    """
    Função principal que executa o script de conversão de arquivos para SHA-256.
    """
    # Solicita o nome do arquivo ao usuário
    nome_arquivo = input("Digite o nome do arquivo (com extensão): ")

    # Verifica se o arquivo tem uma extensão válida
    if not verificar_arquivo(nome_arquivo):
        return  # Encerra a execução se a extensão for inválida

    try:
        # Pergunta se o arquivo original tem cabeçalho
        tem_cabecalho = input("O arquivo original tem cabeçalho? (s/n): ").strip().lower() == 's'

        # Se o arquivo for CSV, pergunta pelo delimitador
        delimitador = ','  # Delimitador padrão
        if nome_arquivo.endswith('.csv'):
            print("Escolha o delimitador do arquivo CSV:")
            print("1. Vírgula (,)")
            print("2. Ponto e vírgula (;)")
            print("3. Tabulação (\\t)")
            print("4. Pipe (|)")
            opcao_delimitador = input("Digite o número correspondente ao delimitador: ").strip()

            # Mapeia a opção escolhida para o delimitador
            delimitadores = {'1': ',', '2': ';', '3': '\t', '4': '|'}
            if opcao_delimitador not in delimitadores:
                print("Opção de delimitador inválida. Usando vírgula (,) como padrão.")
            else:
                delimitador = delimitadores[opcao_delimitador]

        # Lê o arquivo de acordo com sua extensão
        if nome_arquivo.endswith('.txt'):
            # Para arquivos TXT, lê cada linha como um item
            with open(nome_arquivo, 'r') as file:
                linhas = file.readlines()
            df = pd.DataFrame(linhas, columns=['Original'])  # Cria um DataFrame com uma coluna chamada 'Original'
        elif nome_arquivo.endswith('.csv'):
            # Para arquivos CSV, lê o arquivo com ou sem cabeçalho e com o delimitador escolhido
            df = pd.read_csv(nome_arquivo, header=0 if tem_cabecalho else None, delimiter=delimitador)
        elif nome_arquivo.endswith('.xlsx'):
            # Para arquivos XLSX, lê o arquivo com ou sem cabeçalho
            df = pd.read_excel(nome_arquivo, header=0 if tem_cabecalho else None)

        # Verifica se o arquivo contém apenas uma coluna
        if not verificar_coluna_unica(df):
            return  # Encerra a execução se o arquivo tiver mais de uma coluna

        # Pergunta ao usuário se deseja manter a coluna original
        manter_original = input("Deseja manter a coluna original? (s/n): ").strip().lower() == 's'

        # Calcula o hash SHA-256 para cada linha da coluna original
        df['SHA256'] = df.iloc[:, 0].apply(calcular_sha256)

        # Remove a coluna original se o usuário optar por descartá-la
        if not manter_original:
            df = df.drop(columns=[df.columns[0]])

        # Pergunta ao usuário o formato de saída desejado
        print("Escolha o formato de saída:")
        print("1. TXT")
        print("2. CSV")
        print("3. XLSX")
        opcao_saida = input("Digite o número correspondente ao formato de saída: ").strip()

        # Mapeia a opção escolhida para a extensão do arquivo de saída
        extensoes = {'1': '.txt', '2': '.csv', '3': '.xlsx'}
        if opcao_saida not in extensoes:
            print("Opção de saída inválida.")
            return

        extensao_saida = extensoes[opcao_saida]

        # Pergunta ao usuário se deseja incluir cabeçalho no arquivo de saída
        cabecalho_saida = input("Deseja incluir cabeçalho no arquivo de saída? (s/n): ").strip().lower() == 's'

        # Se o formato de saída for CSV, pergunta pelo delimitador
        if extensao_saida == '.csv':
            print("Escolha o delimitador para o arquivo de saída CSV:")
            print("1. Vírgula (,)")
            print("2. Ponto e vírgula (;)")
            print("3. Tabulação (\\t)")
            print("4. Pipe (|)")
            opcao_delimitador_saida = input("Digite o número correspondente ao delimitador: ").strip()

            # Mapeia a opção escolhida para o delimitador
            delimitadores_saida = {'1': ',', '2': ';', '3': '\t', '4': '|'}
            if opcao_delimitador_saida not in delimitadores_saida:
                print("Opção de delimitador inválida. Usando vírgula (,) como padrão.")
                delimitador_saida = ','
            else:
                delimitador_saida = delimitadores_saida[opcao_delimitador_saida]
        else:
            delimitador_saida = ','  # Delimitador padrão (não usado para TXT e XLSX)

        # Pergunta ao usuário onde deseja salvar o arquivo
        caminho_saida = input("Digite o caminho onde deseja salvar o arquivo (deixe em branco para salvar no diretório atual): ").strip()

        # Define o nome do arquivo de saída
        nome_saida = os.path.splitext(nome_arquivo)[0] + '_sha256' + extensao_saida

        # Combina o caminho de saída com o nome do arquivo
        if caminho_saida:
            nome_saida = os.path.join(caminho_saida, nome_saida)

        # Salva o arquivo no formato escolhido
        if extensao_saida == '.txt':
            df.to_csv(nome_saida, index=False, header=cabecalho_saida)  # Salva como TXT com ou sem cabeçalho
        elif extensao_saida == '.csv':
            df.to_csv(nome_saida, index=False, header=cabecalho_saida, sep=delimitador_saida)  # Salva como CSV com ou sem cabeçalho e delimitador escolhido
        elif extensao_saida == '.xlsx':
            df.to_excel(nome_saida, index=False, header=cabecalho_saida)  # Salva como XLSX com ou sem cabeçalho

        # Informa ao usuário que o arquivo foi salvo com sucesso
        print(f"Arquivo processado e salvo como {nome_saida}")

    except Exception as e:
        # Captura e exibe qualquer erro que ocorrer durante a execução
        print(f"Ocorreu um erro: {e}")

# Executa a função principal se o script for rodado diretamente
if __name__ == "__main__":
    main()